In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import torchvision
import os
import time

In [3]:
from models import FireClassifier, BACKBONES, transform
from utils import accuracy_gpu

# Load saved model

In [6]:
!ls ../weights

'densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt'
'densenet121-epoch-1-val_acc=0.9889-test_acc=-1.00.pt'
'densenet121-epoch-2-val_acc=0.9881-test_acc=-1.00.pt'
'densenet121-epoch-3-val_acc=0.9899-test_acc=-1.00.pt'
'densenet121-epoch-4-val_acc=0.9953-test_acc=-1.00.pt'
'densenet121-epoch-5-val_acc=0.9956-test_acc=-1.00.pt'
'densenet121-epoch-6-val_acc=0.9956-test_acc=-1.00.pt'
'densenet121-epoch-7-val_acc=0.9960-test_acc=-1.00.pt'
'densenet121-epoch-8-val_acc=0.9948-test_acc=-1.00.pt'
'densenet121-epoch-9-val_acc=0.9960-test_acc=-1.00.pt'
'resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt'
'resnet101-epoch-1-val_acc=0.9869-test_acc=-1.00.pt'
'resnet101-epoch-2-val_acc=0.9842-test_acc=-1.00.pt'
'resnet101-epoch-3-val_acc=0.9833-test_acc=-1.00.pt'
'resnet101-epoch-4-val_acc=0.9929-test_acc=-1.00.pt'
'resnet101-epoch-5-val_acc=0.9953-test_acc=-1.00.pt'
'resnet101-epoch-6-val_acc=0.9929-test_acc=-1.00.pt'
'resnet101-epoch-7-val_acc=0.9929-test_acc=-1.00.pt'
'resnet101-epoch-8-val_a

In [8]:
model_files = [
    'densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt',
    'resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt',
    'resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt',
    'resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt',
    'resnet50-epoch-10-val_acc=0.9949-test_acc=-1.00.pt',
    'VGG16-epoch-1-val_acc=0.9270-test_acc=-1.00.pt'
]

# Define datasets

In [89]:
dataset_paths = {
    "afd_train": "/media/tomek/BIG2/datasets/FIRE/aerial_fire_dataset/train",
    "afd_test": "/media/tomek/BIG2/datasets/FIRE/aerial_fire_dataset/test/",
    "dunnings_train": "/media/tomek/BIG2/datasets/FIRE/dunnings/fire-dataset-dunnings/images-224x224/train",
    "dunnings_test": "/media/tomek/BIG2/datasets/FIRE/dunnings/fire-dataset-dunnings/images-224x224/test",
    "combined_train": "/media/tomek/BIG2/datasets/FIRE/combined_dunnings_afd/train",
    "combined_test": "/media/tomek/BIG2/datasets/FIRE/combined_dunnings_afd/test"
}

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.4005, 0.3702, 0.3419), std=(0.2858, 0.2749, 0.2742)),
    ]
)

test_dunnings = torchvision.datasets.ImageFolder(root=dataset_paths['dunnings_test'],
                                                 transform=transform)

test_dunnings_loader = torch.utils.data.DataLoader(
    test_dunnings,
    batch_size=8,
    num_workers=0,
    shuffle=False
)

test_afd = torchvision.datasets.ImageFolder(root=dataset_paths['afd_test'],
                                            transform=transform)

test_afd.class_to_idx = {'positive': 1, 'negative': 0} # ensure class mapping
test_afd_loader = torch.utils.data.DataLoader(
    test_afd,
    batch_size=8,
    num_workers=0,
    shuffle=True
)

In [11]:
def average(l):
    return sum(l)/len(l)

In [98]:
device = torch.device("cuda:0")
results = {}
Ys = []

with torch.no_grad():
    
    for fname in model_files:
        model = torch.load(f"../weights/{fname}")
        model = model.to(device)
        model.eval()
        
        Y = []
        P = []
    
        for i, data in enumerate(test_afd_loader):
            print(i)
            inputs = data[0].to(device)
            labels = data[1].to(device)

            scores = model(inputs)
            
            scores = scores.squeeze().to('cpu').tolist()
            labels = labels.squeeze().to('cpu').tolist()
            
            
            if isinstance(scores,float):
                labels = [labels]
                scores = [scores]
             
            for p in scores:
                P.append(p)
                
            for y in labels:
                Y.append(y)

        results[fname] = {'P': P, 'Y': Y}
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [99]:
for k,v in results.items():
    P = v['P']
    Y = v['Y']
    print(f"{k} accuracy: {sklearn.metrics.accuracy_score(Y,np.array(P) < 0.5)}")
    print(f"{k} recall: {sklearn.metrics.recall_score(Y,np.array(P) < 0.5)}")
    print(f"{k} precision: {sklearn.metrics.precision_score(Y,np.array(P) < 0.5)}")
    

densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt accuracy: 0.9701492537313433
densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt recall: 0.94
densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt precision: 1.0
resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt accuracy: 0.9402985074626866
resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt recall: 0.9
resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt precision: 0.9782608695652174
resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt accuracy: 0.9502487562189055
resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt recall: 0.91
resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt precision: 0.9891304347826086
resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt accuracy: 0.9751243781094527
resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt recall: 0.95
resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt precision: 1.0
resnet50-epoch-10-val_acc=0.9949-test_acc=-1.00.pt accuracy: 0.9601990049751243
resnet50-epoch-10-val_acc=0.9949-test_a

In [104]:
device = torch.device("cuda:0")
results = {}
Ys = []

with torch.no_grad():
    
    for fname in model_files:
        print(f"evaluating {fname}")
        model = torch.load(f"../weights/{fname}")
        model = model.to(device)
        model.eval()
        
        Y = []
        P = []
    
        for i, data in enumerate(test_dunnings_loader):
            inputs = data[0].to(device)
            labels = data[1].to(device)

            scores = model(inputs)
            
            scores = scores.squeeze().to('cpu').tolist()
            labels = labels.squeeze().to('cpu').tolist()
            
            
            if isinstance(scores,float):
                labels = [labels]
                scores = [scores]
             
            for p in scores:
                P.append(p)
                
            for y in labels:
                Y.append(y)

        results[fname] = {'P': P, 'Y': Y}

evaluating densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt
evaluating resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt
evaluating resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt
evaluating resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt
evaluating resnet50-epoch-10-val_acc=0.9949-test_acc=-1.00.pt
evaluating VGG16-epoch-1-val_acc=0.9270-test_acc=-1.00.pt


In [106]:
for k,v in results.items():
    P = v['P']
    Y = v['Y']
    print(f"{k} accuracy: {sklearn.metrics.accuracy_score(Y,np.array(P) > 0.5)}")
    print(f"{k} recall: {sklearn.metrics.recall_score(Y,np.array(P) > 0.5)}")
    print(f"{k} precision: {sklearn.metrics.precision_score(Y,np.array(P) > 0.5)}")

densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt accuracy: 0.9580348004094166
densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt recall: 0.9539094650205762
densenet121-epoch-10-val_acc=0.9952-test_acc=-1.00.pt precision: 0.9453507340946167
resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt accuracy: 0.9112930740361651
resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt recall: 0.9440329218106996
resnet101-epoch-10-val_acc=0.9929-test_acc=-1.00.pt precision: 0.8566094100074683
resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt accuracy: 0.9331286250426476
resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt recall: 0.928395061728395
resnet18-epoch-10-val_acc=0.9945-test_acc=-1.00.pt precision: 0.9118835893290218
resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt accuracy: 0.925281473899693
resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt recall: 0.8600823045267489
resnet34-epoch-10-val_acc=0.9933-test_acc=-1.00.pt precision: 0.9552102376599635
resnet50-epoch-10-val_acc=0.9949-t

# AFD test set